<a href="https://colab.research.google.com/github/Hritik1100/Data-Augmentation-in-ML/blob/main/Word2Vec(all).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Only needs to be run once at the top of the notebook
!pip install spacy
!python -m spacy download en_core_web_lg

In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
import pandas as pd
import re
import nltk
import sklearn
import spacy
data = pd.read_csv('newfile.csv',encoding='latin-1')

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

In [ ]:
!pip install git+https://github.com/casics/spiral.git
!pip install textaugment
!pip install contractions
!pip install -U git+https://github.com/ray-project/tune-sklearn.git && pip install 'ray[tune]'
!!pip install scikit-optimize

In [ ]:
from textaugment import Wordnet
t = Wordnet()
aug = t.augment("The quick brown fox jumps over the lazy dog")
print(aug)
print(type(aug))

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
lem = WordNetLemmatizer()
corpus_test = []
stopwords = set(stopwords.words('english'))
punc = string.punctuation

In [ ]:
corpus=[]
import contractions
from spiral import ronin
for i in range(len(data)):
  #review = re.sub(r"http\S+", "", data['Tweets'][i])
  #review = re.sub('[^a-zA-Z]',' ',review)
  review = re.sub(r"http\S+", "", data['message'][i]) # remove urls
  review = re.sub(r'<([^>]*)>', ' ', review) # remove emojis
  review = re.sub(r'@\w+', ' ', review) # remove at mentions
  review = re.sub(r'#', '', review) # remove hashtag symbol
  review = re.sub(r'[0-9]+', ' ', review) # remove numbers
  review = re.sub(r'[^A-Za-z0-9,?.!]+', ' ', review)
  review = review.lower()
  l=[]
  for word in review.split():
    l.append(contractions.fix(word))
  review = ' '.join(l)
  review = t.augment(review)
  review = ronin.split(review)
  review = ' '.join(review)
  review = nltk.word_tokenize(review)
  review = [lem.lemmatize(word) for word in review if word not in stopwords and word not in punc]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
X = np.array([nlp(d).vector for d in corpus])
y = data['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
X_train_vector = X_train
X_test_vector = X_test

In [ ]:
params = {'clf__learning_rate':[0.1,0.3],
          'clf__n_estimators':[70,100]
}


TuneSearch CV


In [ ]:
from sklearn.metrics import precision_score,confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from tune_sklearn import TuneSearchCV
from imblearn.over_sampling import SMOTE           #Run this for Cross Validation after initializing params
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted'),
           'recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')
}

pipeline = Pipeline([('Smote',SMOTE(random_state=42)),('clf',GradientBoostingClassifier())])
tune_imba = TuneSearchCV(pipeline,params,scoring=
                         scoring,max_iters=10,n_jobs=-1,search_optimization = 'bayesian',cv=5,verbose=1,refit='precision')
tune_imba.fit(X_train_vector,y_train)
y_pred = tune_imba.predict(X_test_vector)
print('validation Precision',tune_imba.best_score_)
print("Test precision score - ", precision_score(y_test, y_pred))
print("Test recall score - ", recall_score(y_test, y_pred))
print("Test f1 score - ", f1_score(y_test, y_pred))
print("Test accuracy score - ", accuracy_score(y_test, y_pred))

tune_imba.best_params_

INFO:ray.tune.tune:Total run time: 4930.34 seconds (4930.30 seconds for the tuning loop).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


validation Precision 0.9179314960917162
Test precision score -  0.8417508417508418
Test recall score -  0.8214676889375685
Test f1 score -  0.8314855875831487
Test accuracy score -  0.9250308261405672


{'clf__learning_rate': 0.2483631611523484, 'clf__n_estimators': 86}

In [ ]:
from mlxtend.plotting import plot_learning_curves
import matplotlib.pyplot as plt
from mlxtend.data import iris_data
from mlxtend.preprocessing import shuffle_arrays_unison
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm

pipeline = Pipeline([('Smote',SMOTE(random_state=42)),('clf',ExtraTreesClassifier(max_features='sqrt',min_samples_leaf= 5, n_estimators=111))])
plot_learning_curves(X_train_vector, y_train, X_test_vector, y_test, tune_imba)
#model_pickle = pickle.loads(save_model)plot_learning_curves(X_train_vector, y_train, X_test_vector, y_test, pipeline)
plt.show()

NameError: ignored